In [271]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv


**IMPORT LIBRARIES**

In [272]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.naive_bayes import GaussianNB

In [273]:
train = pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv')
test_data =pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv')
movies = pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv')

In [274]:
train


,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE
...,...,...,...,...,...
162753,the_joker_ethereal_captain_jack_sparrow_fantastic,Danny Mueller,False,A top-notch thriller with genuine surprises an...,POSITIVE
162754,e.t._hannibal_lecter_vito_corleone_katniss_eve...,Jennifer Clayton,True,Some people find Derek Zoolander funny and lik...,NEGATIVE
162755,infinite_enigma_luke_skywalker,Bryan Wilson,False,"This fun, gentle comedy focuses mainly on them...",POSITIVE
162756,emerald_oracle_iron_man_wolverine,Erik Parker,False,"The film is rescued by a strong third act, but...",NEGATIVE


**EDA**

In [275]:
# Available Data
print('---------------')
print('Available Data')
print('Train Data :', train.shape)
print('Test Data :', test_data.shape)
print('Movies Data :', movies.shape)
print('---------------')

---------------
Available Data
Train Data : (162758, 5)
Test Data : (55315, 4)
Movies Data : (143258, 14)
---------------


In [276]:
train.isnull().sum()

movieid                  0
reviewerName             0
isFrequentReviewer       0
reviewText            6447
sentiment                0
dtype: int64

In [277]:
train=train.fillna("NA")
train.shape

(162758, 5)

In [278]:
test_data= test_data.fillna("NA")
test_data.isnull().sum()

movieid         0
reviewerName    0
isTopCritic     0
reviewText      0
dtype: int64

In [279]:
# Average word count
train['reviewText'].apply(lambda x: len(x.split())).mean()

21.013707467528477

In [280]:
train['reviewText'].apply(lambda x: len(x.split())).max()

54

In [281]:
# Remove special characters, links
train['reviewText'] = train['reviewText'].apply(lambda x: re.sub(r'http\S+|www\S+|\W|\d', ' ', x))

In [282]:
test_data['reviewText']


0        Green slowly cranks up the dread with style an...
1        Philip Noyce's direction is elegant and unforc...
2        It wouldn't do to say what path Maria ultimate...
3        Pig is not exactly the arthouse John Wick that...
4        An imaginative no-budget musical of sorts abou...
                               ...                        
55310    Ron Howard delivers an unconventional romantic...
55311    As an oddball art film that openly invites you...
55312    Nicholson wears his devilish grin from his fir...
55313    It's hard not be entertained by two dozen of C...
55314    Not clever enough for Smith fans, not gross en...
Name: reviewText, Length: 55315, dtype: object

In [283]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer()
train_text = vectorizer.fit_transform(train['reviewText'])

test_text = vectorizer.transform(test_data['reviewText'])

In [284]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
train['sentiment']= label_encoder.fit_transform(train['sentiment'])
train_label= train['sentiment']

In [285]:
from sklearn.metrics import accuracy_score, f1_score,make_scorer

In [286]:
 from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_text, train_label, test_size=0.2, random_state=42)

In [287]:
 from sklearn.linear_model import LogisticRegression
logicreg = LogisticRegression(solver='saga',max_iter=1000)

 from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier(max_iter=10000)

 from sklearn.svm import LinearSVC
linearsvc= LinearSVC()

In [288]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier

model_reg= LogisticRegression(solver='saga',max_iter=1000,C=10)
model_ridge=RidgeClassifier(max_iter=10000, alpha=1.0)
model_svm=LinearSVC(C=0.1)

model=VotingClassifier(
    estimators=[
        ('reg', model_reg),
        ('ridge_clf', model_ridge),
        ('svm', model_svm)
    ],
    voting='hard'  
)

In [289]:
model.fit(x_train,y_train)
model_pred= model.predict(x_test)

In [290]:
print('Accuracy: ', accuracy_score(y_test, model_pred))
print('f1_score: ', f1_score(y_test,model_pred))
 

Accuracy:  0.7984148439420005
f1_score:  0.8569185818324538


In [291]:
model.fit(train_text,train_label)

VotingClassifier(estimators=[('reg',
                              LogisticRegression(C=10, max_iter=1000,
                                                 solver='saga')),
                             ('ridge_clf', RidgeClassifier(max_iter=10000)),
                             ('svm', LinearSVC(C=0.1))])

In [292]:
model_pred= model.predict(test_text)

In [293]:
# ypred= model_predict

# submission=pd.DataFrame(columns=['id', 'sentiment'])
# submission['id']=[i for i in range(len(ypred))]
# submission["sentiment"]=[i for i in ypred]
# submission['sentiment']=submission['sentiment']
# submission['sentiment']= submission['sentiment'].replace(to_replace=1, value='POSITIVE')
# submission['sentiment']= submission['sentiment'].replace(to_replace=0, value='NEGATIVE')
# submission.to_csv('submission.csv',index=False)

In [294]:
#print("X_train shape:", x_train.shape)
#print("X_test shape:", x_test.shape)
#print("y_train shape:", y_train.shape)
#print("y_test shape:", y_test.shape)


In [295]:
#a.shape

In [296]:
#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression()
#model.fit(x_train, y_train)

In [297]:
# from sklearn.metrics import f1_score

# y_pred = model.predict(x_test)
# f1 = f1_score(y_test, y_pred)
# print("F1 Score:", f1)

In [298]:
#s =model.predict(a)

In [299]:
#submission=pd.DataFrame(columns=['id', 'sentiment'])
#submission['id']=[i for i in range(len(s))]
#submission["sentiment"]=[i for i in s]
#submission['sentiment'] = submission['sentiment'].apply(lambda a: 'Positive' if a == 1 else 'Negative')
#submission.to_csv('submission.csv',index=False)
#submission

In [300]:
# from sklearn.neighbors import KNeighborsClassifier
# knn_model = KNeighborsClassifier(n_neighbors=3)
# knn_model.fit(x_train, y_train)
# knn_pred = knn_model.predict(x_test)
# knn_accuracy = f1_score(y_test, knn_pred)

In [301]:
#print("KNN Accuracy:", knn_accuracy)

In [302]:
# from sklearn.svm import SVC
# svm_model = SVC()
# svm_model.fit(x_train, y_train)
# svm_pred = svm_model.predict(x_test)
# svm_accuracy = f1_score(y_test, svm_pred)
# print("SVM Accuracy:", svm_accuracy)

In [303]:
#from sklearn.model_selection import GridSearchCV

In [304]:
#model = LogisticRegression(solver='lbfgs', max_iter=1000)
#param_grid = {'C': [0.1, 1.0, 10.0],'penalty': ['l1', 'l2','none']}
#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# grid_search.fit(x, y)

# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# y_pred = best_model.predict(x)
# f1 = f1_score(y, y_pred)
# print("Best Hyperparameters:", best_params)
# print("F1 Score:", f1)

In [305]:
movies.drop_duplicates(subset=['movieid'], keep = 'last', inplace=True)

In [306]:
#Join train table and movies table by "movie_id"
datamix = pd.merge(train,movies, on="movieid", how='left')
testmix= pd.merge(test_data, movies,on ="movieid",how = 'left' )


In [307]:
testmix.rename(columns = {'isTopCritic':'isFrequentReviewer'}, inplace = True)

In [308]:
X = datamix.drop('sentiment', axis= 'columns')
y = datamix['sentiment']

In [309]:
y.shape

(162758,)

In [310]:
X.shape

(162758, 17)

In [311]:
datamix.shape

(162758, 18)

In [312]:
# # Dropping columns with at least 75% missing data
# perc = 75.0
# min_count =  int(((100-perc)/100)*datamix.shape[0] + 1)
# datamix= datamix.dropna( axis=1, thresh=min_count, )
# datamix.shape

In [313]:
#Null values in the dataset
datamix.isnull().sum()

movieid                     0
reviewerName                0
isFrequentReviewer          0
reviewText                  0
sentiment                   0
title                       0
audienceScore           11710
rating                  63219
ratingContents          63219
releaseDateTheaters     34591
releaseDateStreaming    15934
runtimeMinutes           2852
genre                    2378
originalLanguage         3335
director                    0
boxOffice               52322
distributor             40062
soundType               84021
dtype: int64

In [314]:
testmix.isnull().sum()

movieid                     0
reviewerName                0
isFrequentReviewer          0
reviewText                  0
title                       0
audienceScore            4367
rating                  24259
ratingContents          24259
releaseDateTheaters     13448
releaseDateStreaming     5850
runtimeMinutes           1080
genre                     857
originalLanguage         1220
director                    0
boxOffice               19976
distributor             15511
soundType               30177
dtype: int64

**DATA PREPROCESSING**

In [315]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer



In [316]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

In [317]:
numeric_features = ["runtimeMinutes"]
numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

categorical_features = ['movieid','reviewerName','isFrequentReviewer','genre','originalLanguage','director' ]
sentiment_features='reviewText'
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))])
sentiment_transformer=Pipeline(steps=[('tfidf',TfidfVectorizer())])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ('sent',sentiment_transformer,sentiment_features)
    ]
)
        

In [318]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", LogisticRegression(max_iter=10000)) 
          ])

In [319]:
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
mergepredict= clf.predict(testmix )

In [320]:
ypred= mergepredict

submission=pd.DataFrame(columns=['id', 'sentiment'])
submission['id']=[i for i in range(len(ypred))]
submission["sentiment"]=[i for i in ypred]
submission['sentiment']=submission['sentiment']
submission['sentiment']= submission['sentiment'].replace(to_replace=1, value='POSITIVE')
submission['sentiment']= submission['sentiment'].replace(to_replace=0, value='NEGATIVE')
submission.to_csv('submission.csv',index=False)

In [321]:
#count_vectorizer = feature_extraction.text.CountVectorizer()

#example_train_vectors = count_vectorizer.fit_transform(train_data["reviewText"][0:5])

In [322]:
#label_encoder = preprocessing.LabelEncoder()
#train_data['sentiment']= label_encoder.fit_transform(train_data['sentiment'])
#train_data

In [323]:
#print(example_train_vectors[0].todense().shape)
#print(example_train_vectors[0].todense())

In [324]:
#train_vectors = count_vectorizer.fit_transform(train_data["reviewText"])

#test_vectors = count_vectorizer.transform(test_data["reviewText"])

In [325]:
#clf = linear_model.RidgeClassifier()

In [326]:
# Ridge classifier
#scores = model_selection.cross_val_score(clf, train_vectors, train_data["sentiment"], cv=5, scoring="f1")
#scores

In [327]:
#clf.fit(train_vectors, train_data["sentiment"])

In [328]:
#ypred = clf.predict(test_vectors)

In [329]:
#submission=pd.DataFrame(columns=['id', 'sentiment'])
#submission['id']=[i for i in range(len(ypred))]
#submission["sentiment"]=[i for i in ypred]
#submission['sentiment']=submission['sentiment'].astype(bool)
#submission.to_csv('submission.csv',index=False)
#submission

In [330]:
#test =pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv')

#test

In [331]:
#test['reviewText'] = train['reviewText'].fillna('Unknown')

In [332]:
#test.isnull().sum()

In [333]:
#test['reviewText'] = test['reviewText'].apply(lambda x: re.sub(r'http\S+|www\S+|\W|\d', ' ', x))
#x = test['reviewText']